In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select   # seleccion de un dropdown


import time

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


import asyncio

from tqdm.notebook import tqdm
from joblib import Parallel, delayed
from unidecode import unidecode 
import concurrent.futures

In [2]:
#%pip install joblib

In [10]:
import joblib
from joblib import Parallel, delayed

In [14]:
driver.quit()

In [28]:
driver = webdriver.Chrome()

# Open the website
url = 'https://www.mundodeportivo.com/resultados/futbol/laliga'
driver.get(url)



In [29]:
aceptar = driver.find_element(By.XPATH, '//*[@id="didomi-notice-agree-button"]')

aceptar.click()

In [30]:
aceptar_1 = driver.find_element(By.XPATH, '//*[@id="gg-alert-buttons"]/a[2]')

aceptar_1.click()

In [31]:
dropdown = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[1]/div/div[2]')
dropdown.click()

In [32]:
fecha = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[1]/div/div[2]/div/a[2]')
fecha.click()

In [33]:
dropdown_1 = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[1]/div/div[3]')
dropdown_1.click()

In [34]:
jornada = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[1]/div/div[3]/div/a[1]')
jornada.click()

In [22]:
from selenium.common.exceptions import NoSuchElementException  # Add this line


def obtener_datos_jornada(numero_jornada):
    # Inicializar el navegador para cada proceso
    driver = webdriver.Chrome()

    try:
        # Abrir la página principal (reemplaza la URL con la tuya)
        url = "https://www.mundodeportivo.com/resultados/futbol/laliga/2022-2023"
        driver.get(url)

        # intenta aceptar las cookies y otro desplegable
        try:
            aceptar = driver.find_element(By.XPATH, '//*[@id="didomi-notice-agree-button"]')
            aceptar.click()
        except:
            pass
        try:
            aceptar_1 = driver.find_element(By.XPATH, '//*[@id="gg-alert-buttons"]/a[2]')
            aceptar_1.click()
        except:
            pass
        # click en los dropdowns para llegar a donde quiero
        dropdown = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[1]/div/div[3]')
        dropdown.click()
        jornada_selector = driver.find_element(By.XPATH, f'//*[@id="main-container"]/div[3]/div/div[1]/div[1]/div/div[3]/div/a[{numero_jornada}]')
        jornada_selector.click()

        # Esperar a que se carguen los resultados (puedes ajustar el tiempo según sea necesario)
        driver.implicitly_wait(5)

        # crea una lista vacia para meter los datos de las jornadas
        resultados = []
        filas_resultados = driver.find_elements(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[3]')
        
        # itera por cada fila de filas de resultados
        for fila in filas_resultados:
            span_elements = fila.find_elements(By.CLASS_NAME, "match__info-item")
            a_p_elements = fila.find_elements(By.XPATH, "//div/div[1]/div/div[2]/p/a")
            p_a_elements = fila.find_elements(By.XPATH, "//div/div[3]/div/div[2]/p/a")
            golesl_local = fila.find_elements(By.XPATH, "//div/div[1]/div/div[3]/p")
            goles_visitante = fila.find_elements(By.XPATH, "//div/div[3]/div/div[3]/p")
            
            # comprension de la lista para extarer cada elemento de span elemnts
            fechas = [span.text for span in span_elements]
            # comprension de la listapara coger los elementos 0/1/2 de fechas 
            dias = [fecha.split('/')[0] for fecha in fechas if '/' in fecha]
            meses = [fecha.split('/')[1] for fecha in fechas if '/' in fecha]
            años = [fecha.split('/')[2] for fecha in fechas if '/' in fecha]
            
            # creo un diccionario para meter las listas que he iterado y sacado
            data = {
                'dia': dias,
                'mes': meses,
                'año': años,
                'Equipo local': [a_p.text for a_p in a_p_elements],
                'Goles del Equipo Local': [golesl.text for golesl in golesl_local],
                'Goles del Equipo Visitante': [goles.text for goles in goles_visitante],
                'Equipo visitante': [p_a.text for p_a in p_a_elements]
            }
            
            # meto el resultado en la lista resultados
            resultados.append(data)

        return resultados
    except NoSuchElementException as e:
        print(f"Error in obtener_datos_jornada for jornada {numero_jornada}: {e}")
        return []
    finally:
        driver.quit()

# Iterar a través de las jornadas
num_jornadas = 38  # ajusta según el número total de jornadas
# creo una lista vacia para meter todos los datos de todas las jornadas
datos_totales = []

# Use Parallel with error handling
try:
    datos_totales = Parallel(n_jobs=-1, backend="loky")(delayed(obtener_datos_jornada)(j) for j in range(1, num_jornadas + 1))
except Exception as e:
    print(f"An error occurred during parallel execution: {e}")

# Crear el DataFrame para cada jornada
df_list = []

#iteramos por todos los datos de todas las jornadas
for datos_jornada in datos_totales:
    #iteramos por los datos de cada jornada
    for data in datos_jornada:
        # convierte los datos de cada jornada en un dataframe
        df_list.append(pd.DataFrame(data))
        
# me junta todos los dataframes 
if df_list:
    df = pd.concat(df_list, axis=0, ignore_index=True)
else:
    print('hola')

In [27]:
df.head()

,dia,mes,año,Equipo local,Goles del Equipo Local,Goles del Equipo Visitante,Equipo visitante
0,12,8,2022,OSASUNA,2,1,SEVILLA
1,13,8,2022,CELTA DE VIGO,2,2,ESPANYOL
2,13,8,2022,REAL VALLADOLID,0,3,VILLARREAL
3,13,8,2022,BARCELONA,0,0,RAYO VALLECANO
4,14,8,2022,CÁDIZ,0,1,REAL SOCIEDAD


In [47]:
df['fecha'] = df['dia'].astype(str) + '-' + df['mes'].astype(str) + '-' + df['año'].astype(str)
df['fecha'] = pd.to_datetime(df['fecha'], format='%d-%m-%Y')


In [48]:
df.head()

,dia,mes,año,Equipo local,Goles del Equipo Local,Goles del Equipo Visitante,Equipo visitante,fecha
0,12,8,2022,OSASUNA,2,1,SEVILLA,2022-08-12
1,13,8,2022,CELTA DE VIGO,2,2,ESPANYOL,2022-08-13
2,13,8,2022,REAL VALLADOLID,0,3,VILLARREAL,2022-08-13
3,13,8,2022,BARCELONA,0,0,RAYO VALLECANO,2022-08-13
4,14,8,2022,CÁDIZ,0,1,REAL SOCIEDAD,2022-08-14


In [49]:
df = df.drop(['dia', 'mes', 'año'], axis=1)

In [60]:
df.head(100)

,Equipo local,Goles del Equipo Local,Goles del Equipo Visitante,Equipo visitante,fecha
0,Osasuna,2,1,Sevilla,2022-08-12
1,Celta De Vigo,2,2,Espanyol,2022-08-13
2,Real Valladolid,0,3,Villarreal,2022-08-13
3,Barcelona,0,0,Rayo Vallecano,2022-08-13
4,Cadiz,0,1,Real Sociedad,2022-08-14
...,...,...,...,...,...
95,Real Sociedad,1,0,Mallorca,2022-10-19
96,Elche,0,3,Real Madrid,2022-10-19
97,Almeria,3,2,Girona,2022-10-20
98,Osasuna,1,0,Espanyol,2022-10-20


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Equipo local                380 non-null    object        
 1   Goles del Equipo Local      380 non-null    object        
 2   Goles del Equipo Visitante  380 non-null    object        
 3   Equipo visitante            380 non-null    object        
 4   fecha                       380 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 15.0+ KB


In [52]:
df['Goles del Equipo Local'] = pd.to_numeric(df['Goles del Equipo Local'], errors='coerce')
df['Goles del Equipo Visitante'] = pd.to_numeric(df['Goles del Equipo Visitante'], errors='coerce')

In [53]:
df['Equipo local'] = df['Equipo local'].str.lower()
df['Equipo visitante'] = df['Equipo visitante'].str.lower()

In [54]:
df.head()

,Equipo local,Goles del Equipo Local,Goles del Equipo Visitante,Equipo visitante,fecha
0,osasuna,2,1,sevilla,2022-08-12
1,celta de vigo,2,2,espanyol,2022-08-13
2,real valladolid,0,3,villarreal,2022-08-13
3,barcelona,0,0,rayo vallecano,2022-08-13
4,cádiz,0,1,real sociedad,2022-08-14


In [55]:
df['Equipo local'] = df['Equipo local'].str.title()
df['Equipo visitante'] = df['Equipo visitante'].str.title()

In [56]:
df.head()

,Equipo local,Goles del Equipo Local,Goles del Equipo Visitante,Equipo visitante,fecha
0,Osasuna,2,1,Sevilla,2022-08-12
1,Celta De Vigo,2,2,Espanyol,2022-08-13
2,Real Valladolid,0,3,Villarreal,2022-08-13
3,Barcelona,0,0,Rayo Vallecano,2022-08-13
4,Cádiz,0,1,Real Sociedad,2022-08-14


In [57]:
df['Equipo local'] = df['Equipo local'].apply(unidecode)
df['Equipo visitante'] = df['Equipo visitante'].apply(unidecode)

In [58]:
df.head()

,Equipo local,Goles del Equipo Local,Goles del Equipo Visitante,Equipo visitante,fecha
0,Osasuna,2,1,Sevilla,2022-08-12
1,Celta De Vigo,2,2,Espanyol,2022-08-13
2,Real Valladolid,0,3,Villarreal,2022-08-13
3,Barcelona,0,0,Rayo Vallecano,2022-08-13
4,Cadiz,0,1,Real Sociedad,2022-08-14


In [61]:
# añadimos en la lista jornada el numero del lenghth de las jornadas hasta 38
jornadas = []
for i in range(1, 39):  # 38 jornadas
    jornadas.extend([i] * 10)  # 10 partidos en cada jornada

df['Jornada'] = jornadas

In [64]:
df.head(15)

,Equipo local,Goles del Equipo Local,Goles del Equipo Visitante,Equipo visitante,fecha,Jornada
0,Osasuna,2,1,Sevilla,2022-08-12,1
1,Celta De Vigo,2,2,Espanyol,2022-08-13,1
2,Real Valladolid,0,3,Villarreal,2022-08-13,1
3,Barcelona,0,0,Rayo Vallecano,2022-08-13,1
4,Cadiz,0,1,Real Sociedad,2022-08-14,1
5,Valencia Cf,1,0,Girona,2022-08-14,1
6,Almeria,1,2,Real Madrid,2022-08-14,1
7,Athletic Club,0,0,Mallorca,2022-08-15,1
8,Getafe,0,3,Atletico De Madrid,2022-08-15,1
9,Real Betis,3,0,Elche,2022-08-15,1


In [66]:
nuevo_orden = ['Jornada', 'fecha', 'Equipo local', 'Goles del Equipo Local', 'Goles del Equipo Visitante', 'Equipo visitante']
df = df[nuevo_orden]

In [67]:
df.head()

,Jornada,fecha,Equipo local,Goles del Equipo Local,Goles del Equipo Visitante,Equipo visitante
0,1,2022-08-12,Osasuna,2,1,Sevilla
1,1,2022-08-13,Celta De Vigo,2,2,Espanyol
2,1,2022-08-13,Real Valladolid,0,3,Villarreal
3,1,2022-08-13,Barcelona,0,0,Rayo Vallecano
4,1,2022-08-14,Cadiz,0,1,Real Sociedad


In [68]:
#df.to_csv('resultados.csv', index=False)